In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


%load_ext autoreload
%autoreload 2
import preprocessing
import models
import plotting_utils

KeyboardInterrupt: 

In [ ]:
# Suppress all warnings globally (adjust based on specific warnings you want to ignore)
# Suppress convergence warnings
warnings.filterwarnings('ignore', category=ConvergenceWarning)

# Suppress user warnings (such as inappropriate l1_ratio usage)
warnings.filterwarnings('ignore', category=UserWarning)
#warnings.filterwarnings("ignore", message=".*l1_ratio parameter is only used when penalty is 'elasticnet'")

In [ ]:

# Example usage:
param_grid_rf = {
    'n_estimators': [100, 500, 1000, 5000],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

In [ ]:
CNA_file = '/Users/irf3irf3/Desktop/offline_workspace/Jessica/Jessica_Model code & inputs/CNA_no_trim_low_1Mb_irf.tsv'
Ratio_file = '/Users/irf3irf3/Desktop/offline_workspace/Jessica/Jessica_Model code & inputs/ratios_150_ratio.centered_irf.tsv' ## for non serial files
EndMotif_file_with_rest_5 = '/Users/irf3irf3/Desktop/offline_workspace/data/tissue_of_origin/EndMotif_frequency_withRest5.txt'
EndMotif_file = '/Users/irf3irf3/Desktop/offline_workspace/data/tissue_of_origin/Not Normalized/EndMotif_frequency_ALL.txt'
Output_dir = '/Users/irf3irf3/Desktop/offline_workspace/data/tissue_of_origin/ML_output_dissertation_beforeLOD'

CNA = pd.read_csv(CNA_file, sep='\t', index_col=[0,1,2])
Ratio = pd.read_csv(Ratio_file, sep='\t', index_col=[0,1,2])
EndMotif = pd.read_csv(EndMotif_file, sep='\t', index_col=0)

Endmotif_file_with_rest5_df = pd.read_csv(EndMotif_file_with_rest_5, sep='\t', index_col=0)




print(f'before filtering {EndMotif.shape}')
# Filter the columns in the first file
EndMotif = EndMotif[Endmotif_file_with_rest5_df.columns]

print(f'after filtering {EndMotif.shape}')

LOOCV_summary = {}
Train_test_summary = {}



display(CNA.head())
display(Ratio.head())
display(EndMotif.head())

In [ ]:
EndMotif.columns = EndMotif.columns.str.replace('_sorted_motifs.txt', '')
EndMotif.columns = EndMotif.columns.str.replace('.dup_mk_motifs.txt', '')
EndMotif.columns = EndMotif.columns.str.replace('_motifs.txt', '')
display(EndMotif.head())

In [ ]:
# Step 1: Find common columns
common_columns = set(EndMotif.columns) & set(CNA.columns) & set(Ratio.columns)
common_columns_list = list(common_columns)  # Convert to list

# Step 2: Find non-common columns
non_common_endmotif = set(EndMotif.columns) - common_columns
non_common_cna = set(CNA.columns) - common_columns
non_common_ratio = set(Ratio.columns) - common_columns

# Print non-common columns
print("Non-common columns in EndMotif:", non_common_endmotif)
print("Non-common columns in CNA:", non_common_cna)
print("Non-common columns in Ratio:", non_common_ratio)

# Step 3: Filter DataFrames to keep only common columns
EndMotif = EndMotif[common_columns_list]
CNA = CNA[common_columns_list]
Ratio = Ratio[common_columns_list]


print(CNA.shape)
display(CNA.head())

print(Ratio.shape)
display(Ratio.head())

print(EndMotif.shape)
display(EndMotif.head())

In [ ]:
# Sort the common columns list alphabetically
common_columns_list = sorted(list(common_columns))

# Reassign the DataFrames using the sorted common columns list
EndMotif = EndMotif[common_columns_list]
CNA = CNA[common_columns_list]
Ratio = Ratio[common_columns_list]


print(CNA.shape)
display(CNA.head())

print(Ratio.shape)
display(Ratio.head())

print(EndMotif.shape)
display(EndMotif.head())

In [ ]:
CNA_t  = preprocessing.preprocess_dataframe(CNA)
display(CNA_t.head(n=10))

In [ ]:
Ratio_t = preprocessing.preprocess_dataframe(Ratio)
display(Ratio_t.head(n=10))

In [ ]:
EndMotif_t = preprocessing.preprocess_dataframe(EndMotif)
display(EndMotif_t.head(n=10))

# Data preparation

In [ ]:
CNA_t = preprocessing.remove_nan_inf_columns(CNA_t)
Ratio_t = preprocessing.remove_nan_inf_columns(Ratio_t)
EndMotif_t = preprocessing.normalize_features_by_sample(preprocessing.remove_nan_inf_columns(EndMotif_t))

display(CNA_t.head(n=10))
display(Ratio_t.head(n=10))
display(EndMotif_t.head(n=10))

In [ ]:


CNA_t_unique = preprocessing.make_value_unique(CNA_t, 0.015625)
display(CNA_t_unique.head(n=10))

In [ ]:
display(CNA_t_unique.describe())
display(Ratio_t.describe())
display(EndMotif_t.describe())

# ML model

In [ ]:

preprocessing.plot_class_distribution(EndMotif_t)

In [ ]:


TEST_SIZE = 0.3
CNA_train, CNA_test = preprocessing.stratified_train_test_split(CNA_t_unique, test_size=TEST_SIZE)

Ratio_train, Ratio_test = preprocessing.stratified_train_test_split(Ratio_t, test_size=TEST_SIZE)

EndMotif_train, EndMotif_test = preprocessing.stratified_train_test_split(EndMotif_t, test_size=TEST_SIZE)




In [ ]:
EndMotif_rf = models.train_model(EndMotif_train, EndMotif_test)

In [ ]:
Ratio_rf = models.train_model(Ratio_train, Ratio_test)

In [ ]:
CNA_rf = models.train_model(CNA_train, CNA_test)

# Meta Model

## Combine all features to a single dataframe

In [ ]:

CNA_scaled = preprocessing.standardize_dataframe(CNA_t_unique)
Ratio_scaled = preprocessing.standardize_dataframe(Ratio_t)
EndMotif_scaled = preprocessing.standardize_dataframe(EndMotif_t)





# CNA_train, CNA_test = preprocessing.stratified_train_test_split(CNA_t_unique, test_size=TEST_SIZE)

# Ratio_train, Ratio_test = preprocessing.stratified_train_test_split(Ratio_t, test_size=TEST_SIZE)

# EndMotif_train, EndMotif_test = preprocessing.stratified_train_test_split(EndMotif_t, test_size=TEST_SIZE)

display(CNA_scaled.head())

In [ ]:
preprocessing.check_scaling(CNA_scaled)

In [ ]:
preprocessing.check_scaling(EndMotif_scaled)

In [ ]:
import preprocessing

combined_df = preprocessing.combine_feature_dfs_with_target([(CNA_scaled, 'CNA_'), (Ratio_scaled, 'Ratio_'), (EndMotif_scaled, 'EndMotif_')])

display(combined_df)

In [ ]:
# Assuming 'combined_df' is the DataFrame you've generated
# Split the DataFrame using the predefined function
train_df, test_df = preprocessing.stratified_train_test_split(combined_df, test_size=TEST_SIZE)

# Extract the feature subsets by prefix from train_df and test_df
CNA_train_df = preprocessing.filter_columns_by_prefix(train_df, 'CNA_')
Ratio_train_df = preprocessing.filter_columns_by_prefix(train_df, 'Ratio_')
EndMotif_train_df = preprocessing.filter_columns_by_prefix(train_df, 'EndMotif_')

CNA_test_df = preprocessing.filter_columns_by_prefix(test_df, 'CNA_')
Ratio_test_df = preprocessing.filter_columns_by_prefix(test_df, 'Ratio_')
EndMotif_test_df = preprocessing.filter_columns_by_prefix(test_df, 'EndMotif_')



In [ ]:
display(EndMotif_train_df)

In [ ]:
display(CNA_train_df)

In [ ]:
# # Train the models on their respective feature subsets using your train_model( function
from sklearn.ensemble import RandomForestClassifier
model_CNA,CNA_RF_auc = models.train_model(CNA_train_df, CNA_test_df, model = RandomForestClassifier(random_state=0, n_estimators=2000, class_weight='balanced'), param_grid=param_grid_rf, cv=5, search_method='grid',
                               save_figures_path=Output_dir+'_/RF_CNA_ROC', 
                               save_folder=Output_dir+'_/RF_CNA_ROC'       )
model_Ratio,Ratio_RF_auc = models.train_model(Ratio_train_df, Ratio_test_df,  model = RandomForestClassifier(random_state=0, n_estimators=2000, class_weight='balanced'), param_grid=param_grid_rf, cv=5,  search_method='grid',
                                 save_figures_path=Output_dir+'_/RF_Ratio_ROC', 
                                 save_folder=Output_dir+'_/RF_Ratio_ROC'       
                                
                                )
model_EndMotif,EndMotif_RF_auc = models.train_model(EndMotif_train_df, EndMotif_test_df,  model = RandomForestClassifier(random_state=0, n_estimators=2000, class_weight='balanced'), param_grid=param_grid_rf, cv=5,search_method='grid',
                               save_figures_path=Output_dir+'_/RF_EndMotif_ROC', 
                               save_folder=Output_dir+'_/RF_EndMotif_ROC'
                                    
                                   )


Train_test_summary['CNA_RF_auc'] = CNA_RF_auc
Train_test_summary['Ratio_RF_auc'] = Ratio_RF_auc
Train_test_summary['EndMotif_RF_auc'] = EndMotif_RF_auc

Train_test_summary



In [ ]:
models.train_model(train_df, test_df)

In [ ]:
param_grid_log_reg = [
    {
        'penalty': ['l2'],         
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['lbfgs'],       
        'max_iter': [200, 500, 1000]
    },
    {
        'penalty': ['l1', 'l2'],   
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['liblinear'],   
        'max_iter': [200, 500, 1000]
    },
    {
        'penalty': ['elasticnet'],  # Only include 'elasticnet' here
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['saga'],         
        'max_iter': [200, 500, 1000],
        'l1_ratio': [0.5]           # Only used with 'elasticnet'
    }
]





#log_reg_model = LogisticRegression(max_iter=1000, random_state=0, class_weight='balanced')




# Train with LogisticRegression and hyperparameter tuning using GridSearchCV
trained_log_reg_model,_ = models.train_model(
        train_df, 
        test_df, 
        target_name='target', 
        model=LogisticRegression(random_state=0, class_weight='balanced'), 
        param_grid=param_grid_log_reg, 
        cv=5, 
        scoring='roc_auc_micro',
        search_method='grid'
)



In [ ]:
param_grid_svm = {
    'C': [0.01, 0.1, 1, 10, 100],  # Small to large values to balance underfitting/overfitting
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],  # Default + specific values
    'kernel': ['rbf']  # Keeping RBF fixed (if you want to try others, add 'linear', 'poly', etc.)
}

models.train_model(
    train_df, 
    test_df, 
    target_name='target', 
    model=SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=0), 
    param_grid=param_grid_svm,  # Ensure you define param_grid for SVM
    cv=5, 
    scoring='roc_auc_micro',
    search_method='grid'
)

In [ ]:





_,CNA_LogReg_auc = models.train_model(CNA_train_df, CNA_test_df, model=LogisticRegression(random_state=0, class_weight='balanced'), 
            param_grid=param_grid_log_reg, 
            cv=5, 
        
            search_method='grid',

            save_figures_path=Output_dir+'_/Log_reg_CNA_ROC', 
            save_folder=Output_dir+'_/Log_reg_CNA_ROC'       
                  )
    

Train_test_summary['CNA_LogReg_auc'] = CNA_LogReg_auc

In [ ]:
_,CNA_SVM_auc = models.train_model(
    CNA_train_df, CNA_test_df, 
    target_name='target', 
    model=SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=0), 
    param_grid=param_grid_svm,  # Ensure you define param_grid for SVM
    cv=5, 
    
    search_method='grid',

    save_figures_path=Output_dir+'_/SVC_CNA_ROC', 
    save_folder=Output_dir+'_/SVC_CNA_ROC'       
    
)

Train_test_summary['CNA_SVM_auc'] = CNA_SVM_auc


In [ ]:


Ratio_logreg_trained_model, Ratio_LogReg_auc = models.train_model(Ratio_train_df, Ratio_test_df, model=LogisticRegression(random_state=0, class_weight='balanced'), 
            param_grid=param_grid_log_reg, 
            cv=5, 

            search_method='grid',
            save_figures_path=Output_dir+'_/Log_reg_Ratio_ROC', 
            save_folder=Output_dir+'_/Log_reg_Ratio_ROC'       
                  )

Train_test_summary['Ratio_LogReg_auc'] = Ratio_LogReg_auc

In [ ]:
_, Ratio_SVM_auc = models.train_model(
    Ratio_train_df, Ratio_test_df,
    target_name='target', 
    model=SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=0), 
    param_grid=param_grid_svm,  # Ensure you define param_grid for SVM
    cv=5, 
 
    search_method='grid',
    save_figures_path=Output_dir+'_/SVC_Ratio_ROC', 
    save_folder=Output_dir+'_/SVC_Ratio_ROC'       
)

Train_test_summary['Ratio_SVM_auc'] = Ratio_SVM_auc

In [ ]:
EndMotif_logreg_trained_model, EndMotif_LogReg_auc= models.train_model(EndMotif_train_df, EndMotif_test_df, model=LogisticRegression(random_state=0, class_weight='balanced'), 
            param_grid=param_grid_log_reg, 
            cv=5, 
          
            search_method='grid',
            save_auc = Output_dir+'_/Log_reg_Endmotif_ROC',
            save_figures_path=Output_dir+'_/Log_reg_EndMotif_ROC', 
            save_folder=Output_dir+'_/Log_reg_EndMotif_ROC'       
    )

Train_test_summary['EndMotif_LogReg_auc'] = EndMotif_LogReg_auc

In [ ]:
_,EndMotif_SVM_auc = models.train_model(
    EndMotif_train_df, EndMotif_test_df,
    target_name='target', 
    model=SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=0), 
    param_grid=param_grid_svm,  # Ensure you define param_grid for SVM
    cv=5, 
   
    search_method='grid',
    save_figures_path=Output_dir+'_/SVC_EndMotif_ROC', 
        save_folder=Output_dir+'_/SVC_EndMotif_ROC'       
)

Train_test_summary['EndMotif_SVM_auc'] = EndMotif_SVM_auc

In [ ]:
base_models = [model_CNA, model_Ratio, model_EndMotif]
train_dfs = [CNA_train_df, Ratio_train_df, EndMotif_train_df]
test_dfs = [CNA_test_df, Ratio_test_df, EndMotif_test_df]

meta_classifier = models.train_meta_classifier(base_models, train_dfs, test_dfs)

In [ ]:
# from xgboost import XGBClassifier
# # Define your meta-classifier as XGBoost
# meta_classifier = XGBClassifier(random_state=0, n_estimators=100, learning_rate=0.1)

# Assuming base_models, train_dfs, and test_dfs are already defined
meta_classifier, y_meta_test_pred_decoded, y_meta_test = models.train_xgboost_meta_classifier(base_models, train_dfs, test_dfs)

# Output the true labels and the predicted labels
print("True labels:\n", y_meta_test.value_counts())
print("Predicted labels:\n", pd.Series(y_meta_test_pred_decoded).value_counts())

In [ ]:
# model_CNA = models.train_model(CNA_train_df, CNA_test_df, model = RandomForestClassifier(random_state=0, n_estimators=2000, class_weight='balanced'), param_grid=param_grid_rf, cv=5, scoring='roc_auc_micro', search_method='grid')
# model_Ratio = models.train_model(Ratio_train_df, Ratio_test_df,  model = RandomForestClassifier(random_state=0, n_estimators=2000, class_weight='balanced'), param_grid=param_grid_rf, cv=5, scoring='roc_auc_micro', search_method='grid')
# model_EndMotif = models.train_model(EndMotif_train_df, EndMotif_test_df,  model = RandomForestClassifier(random_state=0, n_estimators=2000, class_weight='balanced'), param_grid=param_grid_rf, cv=5, scoring='roc_auc_micro', search_method='grid')



EndMotif_feature_importance = preprocessing.extract_feature_importances(model_EndMotif, EndMotif_train_df)

CNA_feature_importance = preprocessing.extract_feature_importances(model_CNA, CNA_train_df)

Ratio_feature_importance = preprocessing.extract_feature_importances(model_Ratio, Ratio_train_df)


In [ ]:

print(EndMotif_feature_importance)

print(CNA_feature_importance)

print(Ratio_feature_importance)

In [ ]:
feature_importances_dict = {
    'EndMotif': EndMotif_feature_importance,
    'CNA': CNA_feature_importance,
    'Ratio': Ratio_feature_importance
}

train_subset, test_subset = preprocessing.subset_top_k_features(train_df, test_df, k=256, feature_importances_dict=feature_importances_dict, target_name='target')

display(train_subset.head())
display(test_subset.head())

In [ ]:
# Train with LogisticRegression and hyperparameter tuning using GridSearchCV
trained_log_reg_model,_ = models.train_model(
        train_subset, 
        test_subset, 
        target_name='target', 
        model=LogisticRegression(random_state=0, class_weight='balanced'), 
        param_grid=param_grid_log_reg, 
        cv=5, 
        scoring='roc_auc_macro',
        search_method='grid',
        save_auc = Output_dir+'_/pan_fature_ROC'
)

In [ ]:
# import joblib

# # Save the model to a file
# joblib.dump(trained_log_reg_model, Output_dir+'/trained_log_reg_model.pkl')

# train_df.to_csv(Output_dir+"/train_df.txt",sep='\t')
# test_df.to_csv(Output_dir+"/test_df.txt",sep='\t')
# train_subset.to_csv(Output_dir+"/train_subset.txt",sep='\t')
# test_subset.to_csv(Output_dir+"/train_subset.txt",sep='\t')

# LOOCV

In [ ]:
# from sklearn.datasets import load_iris
# from sklearn.model_selection import train_test_split
# import pandas as pd

# # Load the Iris dataset
# iris = load_iris()

# # Create a DataFrame
# data_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
# data_iris['target'] = iris.target  # Add target column

# # Split the dataset into train and test sets
# iris_train_df, iris_test_df = train_test_split(data_iris, test_size=0.2, random_state=0, stratify=data_iris['target'])


# iris_dataset = models.combine_train_test(iris_train_df, iris_test_df)
# # Test the train_model_loocv function
# iris_model, iris_loocv_accuracy = models.train_model_loocv(iris_dataset, target_name='target')

# # Print the results
# print(f"Final model trained on train_df with LOOCV Accuracy: {iris_loocv_accuracy:.4f}")


In [ ]:
# # Load the Breast Cancer dataset
# from sklearn.datasets import load_breast_cancer
# from sklearn.model_selection import train_test_split
# import pandas as pd

# # Load the Breast Cancer dataset
# breast = load_breast_cancer()

# # Create a DataFrame
# data_breast = pd.DataFrame(breast.data, columns=breast.feature_names)
# data_breast['target'] = breast.target  # Add target column

# # Replace numeric target with descriptive labels
# data_breast['target'] = data_breast['target'].map({0: 'malignant', 1: 'benign'})

# # Split the dataset into train and test sets
# breast_train_df, breast_test_df = train_test_split(data_breast, test_size=0.2, random_state=0, stratify=data_breast['target'])

# # Combine train and test sets for LOOCV
# breast_dataset = models.combine_train_test(breast_train_df, breast_test_df)

# # Test the train_model_loocv function
# breast_model, breast_loocv_accuracy = models.train_model_loocv(breast_dataset, target_name='target')

# # Print the results
# print(f"Final model trained on train_df with LOOCV Accuracy: {breast_loocv_accuracy:.4f}")


In [ ]:
import pandas as pd

def subset_rows_by_target(df, target_values):
    """
    Subset rows of a DataFrame based on specific values in the 'target' column.

    Parameters:
    - df (pd.DataFrame): The DataFrame to subset.
    - target_values (list): A list of values to keep in the 'target' column.

    Returns:
    - pd.DataFrame: A new DataFrame with rows where 'target' is in target_values.
    """
    return df[df['target'].isin(target_values)]

# Example usage:
# data = {'target': ['Healthy', 'Bladder', 'Cancer', 'Healthy', 'Bladder'],
#         'value': [10, 20, 30, 40, 50]}
# df = pd.DataFrame(data)
# filtered_df = subset_rows_by_target(df, ['Healthy', 'Bladder'])
# print(filtered_df)


In [ ]:


# Combine train and test sets for LOOCV
EndMotif_dataset = models.combine_train_test(EndMotif_train_df, EndMotif_test_df)

# Test the train_model_loocv function
#EndMotif_LOOCV_model, EndMotif_LOOCV_accuracy = 
EndMotif_LogReg_LOOCV_model,  EndMotif_LogReg_LOOCV_auc, EndMotif_LogReg_oof_df=models.train_model_loocv(EndMotif_dataset, target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced'),save_figures_path=Output_dir+'_/LogReg_Endmotif_LOOCV_ROC', save_folder=Output_dir+'_/LogReg_Endmotif_LOOCV_ROC')

# Print the results
# print(f"Final model trained on train_df with LOOCV Accuracy: {EndMotif_LOOCV_accuracy:.4f}")

display(EndMotif_LogReg_oof_df.head())

LOOCV_summary['EndMotif_LogReg_LOOCV_auc'] = EndMotif_LogReg_LOOCV_auc


LOOCV_summary

In [ ]:
# Test the train_model_loocv function
_,EndMotif_RF_LOOCV_auc,_=models.train_model_loocv(EndMotif_dataset, target_name='target',model=RandomForestClassifier(random_state=0,n_estimators=2000, class_weight='balanced'),save_figures_path=Output_dir+'_/RF_Endmotif_LOOCV_ROC', save_folder=Output_dir+'_/RF_Endmotif_LOOCV_ROC')

LOOCV_summary['EndMotif_RF_LOOCV_auc'] = EndMotif_RF_LOOCV_auc

In [ ]:
_,EndMotif_SVM_LOOCV_auc,_ = models.train_model_loocv(EndMotif_dataset, target_name='target',
    model=SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, class_weight='balanced'), 
    save_figures_path=Output_dir+'_/SVM_EndMotif_LOOCV_ROC', 
    save_folder=Output_dir+'_/SVM_EndMotif_LOOCV_ROC')


LOOCV_summary['EndMotif_SVM_LOOCV_auc'] = EndMotif_SVM_LOOCV_auc

In [ ]:
Ratio_dataset = models.combine_train_test(Ratio_train_df, Ratio_test_df)
# Test the train_model_loocv function
_, Ratio_LogReg_LOOCV_auc, Ratio_LogReg_oof_df = models.train_model_loocv(Ratio_dataset, target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced'),save_figures_path=Output_dir+'_/LogReg_Ratio_LOOCV_ROC', save_folder=Output_dir+'_/LogReg_Ratio_LOOCV_ROC')


LOOCV_summary['Ratio_LogReg_LOOCV_auc'] = Ratio_LogReg_LOOCV_auc

# Print the results
#print(f"Final model trained on train_df with LOOCV Accuracy: {Ratio_LOOCV_accuracy:.4f}")

In [ ]:
_,Ratio_RF_LOOCV_auc,_ = models.train_model_loocv(Ratio_dataset, target_name='target',model=RandomForestClassifier(random_state=0,n_estimators=2000, class_weight='balanced'),save_figures_path=Output_dir+'_/RF_Ratio_LOOCV_ROC', save_folder=Output_dir+'_/RF_Ratio_LOOCV_ROC')


LOOCV_summary['Ratio_RF_LOOCV_auc'] = Ratio_RF_LOOCV_auc

In [ ]:
_,Ratio_SVM_LOOCV_auc,_= models.train_model_loocv(Ratio_dataset, target_name='target',
    model=SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, class_weight='balanced'), 
    save_figures_path=Output_dir+'_/SVM_Ratio_LOOCV_ROC', 
    save_folder=Output_dir+'_/SVM_Ratio_LOOCV_ROC')

LOOCV_summary['Ratio_SVM_LOOCV_auc'] = Ratio_SVM_LOOCV_auc

In [ ]:
Arm_file = '/Users/irf3irf3/Desktop/offline_workspace/Jessica/R code for bladder vs. healthy models/arm_z_wide.tsv'

Arm_df = pd.read_csv(Arm_file, sep='\t')
Arm_df = Arm_df.set_index('library')
Arm_df.rename(columns={'cohort': 'target'}, inplace=True)
Arm_df = Arm_df[Arm_df['split'].isin(['testing', 'training'])]
Arm_df.index.name = None
Arm_df = Arm_df.drop(columns=['split'])
print(Arm_df.shape)
display(Arm_df.head())

In [ ]:
train_Arm_df = Arm_df.loc[EndMotif_train_df.index]
test_Arm_df = Arm_df.loc[EndMotif_test_df.index]

In [ ]:
train_Arm_df.head()

In [ ]:
print(EndMotif_train_df.shape)
print(EndMotif_test_df.shape)
EndMotif_train_df.head()#, EndMotif_test_df

In [ ]:
_,Arm_SVM_auc = models.train_model(
    train_Arm_df, test_Arm_df,
    target_name='target', 
    model=SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=0), 
    param_grid=param_grid_svm,  # Ensure you define param_grid for SVM
    cv=5, 
    search_method='grid',
    save_figures_path=Output_dir+'_/SVC_Arm_ROC', 
    save_folder=Output_dir+'_/SVC_Arm_ROC'
)

Train_test_summary['Arm_SVM_auc'] = Arm_SVM_auc

In [ ]:
Arm_logreg_trained_model,Arm_LogReg_auc = models.train_model(train_Arm_df, test_Arm_df, model=LogisticRegression(random_state=0, class_weight='balanced'), 
            param_grid=param_grid_log_reg, 
            cv=5, 
   
            search_method='grid',
            save_figures_path=Output_dir+'_/Log_reg_Arm_ROC', 
            save_folder=Output_dir+'_/Log_reg_Arm_ROC'
                   
                  )

Train_test_summary['Arm_LogReg_auc'] = Arm_LogReg_auc

In [ ]:
_,Arm_RF_auc = models.train_model(train_Arm_df, test_Arm_df, model = RandomForestClassifier(random_state=0, n_estimators=2000, class_weight='balanced'), param_grid=param_grid_rf, cv=5, scoring='roc_auc_micro', search_method='grid',
                save_figures_path=Output_dir+'_/RF_Arm_ROC', 
                save_folder=Output_dir+'_/RF_Arm_ROC')



Train_test_summary['Arm_RF_auc'] =Arm_RF_auc

In [ ]:
# Test the train_model_loocv function
_, Arm_LogReg_LOOCV_auc, Arm_LogReg_oof_df= models.train_model_loocv(Arm_df, target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced'), save_figures_path=Output_dir+'_/LogReg_Arm_LOOCV_ROC', save_folder=Output_dir+'_/LogReg_Arm_LOOCV_ROC')


LOOCV_summary['Arm_LogReg_LOOCV_auc'] = Arm_LogReg_LOOCV_auc
# Print the results
#print(f"Final model trained on train_df with LOOCV Accuracy: {Arm_LOOCV_accuracy:.4f}")

In [ ]:
_,Arm_RF_LOOCV_auc,_ = models.train_model_loocv(Arm_df, target_name='target',model=RandomForestClassifier(random_state=0,n_estimators=2000, class_weight='balanced'), save_figures_path=Output_dir+'_/RF_Arm_LOOCV_ROC', save_folder=Output_dir+'_/RF_Arm_LOOCV_ROC')
LOOCV_summary['Arm_RF_LOOCV_auc'] = Arm_RF_LOOCV_auc

In [ ]:
_,Arm_SVM_LOOCV_auc,_ = models.train_model_loocv(
    Arm_df, 
    target_name='target',
    model=SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, class_weight='balanced'), 
    save_figures_path=Output_dir+'_/SVM_Arm_LOOCV_ROC', 
    save_folder=Output_dir+'_/SVM_Arm_LOOCV_ROC'
)

LOOCV_summary['Arm_SVM_LOOCV_auc'] = Arm_SVM_LOOCV_auc

# XGDB meta classifier

In [ ]:
display(Arm_LogReg_oof_df.head())
display(EndMotif_LogReg_oof_df.head())
display(Ratio_LogReg_oof_df.head())

In [ ]:
# Creating copies with prefixed column names
Arm_LogReg_oof_prefix = Arm_LogReg_oof_df.add_prefix("Arm_").copy()
EndMotif_LogReg_oof_prefix = EndMotif_LogReg_oof_df.add_prefix("EndMotif_").copy()
Ratio_LogReg_oof_prefix = Ratio_LogReg_oof_df.add_prefix("Ratio_").copy()

# Concatenating and reindexing
AllLogReg_predictions = pd.concat([Arm_LogReg_oof_prefix, EndMotif_LogReg_oof_prefix, Ratio_LogReg_oof_prefix], axis=1).reindex(Arm_LogReg_oof_prefix.index)


AllLogReg_predictions = AllLogReg_predictions.drop(columns=['Arm_Predicted Label','EndMotif_True Label','EndMotif_Predicted Label','Ratio_True Label','Ratio_Predicted Label'], errors='ignore')

AllLogReg_predictions = AllLogReg_predictions.rename(columns={'Arm_True Label':'target'})
display(AllLogReg_predictions.head())
print(AllLogReg_predictions.shape)
print(Ratio_LogReg_oof_df.shape)

In [ ]:
from xgboost import XGBClassifier
_,XGB_AllthreeLogReg_LOOCV_auc,_=models.train_model_loocv_xgb(
    AllLogReg_predictions, 
    target_name='target',
    model=XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss'),
    save_figures_path=Output_dir+'_/XGB_AllThree_LOOCV_ROC',
    save_folder=Output_dir+'_/XGB_AllThree_LOOCV_ROC'
)

#LOOCV_summary['XGB_AllthreeLogReg_LOOCV_auc'] = XGB_AllthreeLogReg_LOOCV_auc



In [ ]:
train_AllLogReg_predictions_df = AllLogReg_predictions.loc[EndMotif_train_df.index]
test_AllLogReg_predictions_df = AllLogReg_predictions.loc[EndMotif_test_df.index]


display(train_AllLogReg_predictions_df.head())
display(test_AllLogReg_predictions_df.head())
print(train_AllLogReg_predictions_df.shape)
print(test_AllLogReg_predictions_df.shape)

In [ ]:
AllLogReg_predictions_xgb_trained_model, XGB_AllthreeLogReg_auc = models.train_model(train_AllLogReg_predictions_df, test_AllLogReg_predictions_df, model=LogisticRegression(random_state=0, class_weight='balanced'), 
            param_grid=param_grid_log_reg, 
            cv=5, 
   
            search_method='grid',
            save_figures_path=Output_dir+'_/XGB_AllThree_ROC', 
            save_folder=Output_dir+'_/XGB_AllThree_ROC'
                   
                  )

#LOOCV_summary['XGB_AllthreeLogReg_auc'] = XGB_AllthreeLogReg_auc

In [ ]:

columns_to_drop = [col for col in AllLogReg_predictions.columns if col.startswith('Ratio_')]
End_Arm_LogReg_predictions = AllLogReg_predictions.drop(columns=columns_to_drop)
display(End_Arm_LogReg_predictions.head())
print(End_Arm_LogReg_predictions.shape)

In [ ]:
_,_,_ = models.train_model_loocv_xgb(
    End_Arm_LogReg_predictions, 
    target_name='target',
    model=XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss'),
    save_figures_path=Output_dir+'_/XGB_End_Arm_LOOCV_ROC',
    save_folder=Output_dir+'_/XGB_End_Arm_LOOCV_ROC'
)

## Methylation

In [ ]:
Methylation_file = '/Users/irf3irf3/Desktop/offline_workspace/data/Fragma_post_processing/Fragma_Ready_fragma_all_combined_all_files_zeroCols_17/Fragma_Ready_fragma_all_combined_all_files_zeroCols_17.txt_MLready.txt'
Methylation = pd.read_csv(Methylation_file, sep= '\t',index_col = 0)
display(Methylation.head())

In [ ]:

Methylation_common_columns = Methylation.columns.intersection(common_columns_list)
Methylation = Methylation[Methylation_common_columns]
print(Methylation.shape)
display(Methylation.head())

In [ ]:
print(Methylation_common_columns.tolist())

In [ ]:
Methylation = preprocessing.preprocess_dataframe(Methylation)
display(Methylation.head())

In [ ]:
Methylation_standardized = preprocessing.standardize_dataframe(Methylation)
display(Methylation_standardized.head())

In [ ]:
# Test the train_model_loocv function
_,_,_ = models.train_model_loocv(Methylation, target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced')) 

# Print the results
#print(f"Final model trained on train_df with LOOCV Accuracy: {Methylation_LOOCV_accuracy:.4f}")

In [ ]:
# Test the train_model_loocv function
_, _,_ = models.train_model_loocv(Methylation_standardized, target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced')) 

# Print the results
#print(f"Final model trained on train_df with LOOCV Accuracy: {Methylation_standardized_LOOCV_accuracy:.4f}")

# Binary: Bladder vs Healthy

In [ ]:
target_values = ['Bladder','Healthy']

In [ ]:
# Combine train and test sets for LOOCV
EndMotif_dataset_binary = subset_rows_by_target(EndMotif_dataset, target_values)

# Test the train_model_loocv function
EndMotif_binary_LOOCV_model, EndMotif_binary_LOOCV_accuracy,_ = models.train_model_loocv(EndMotif_dataset_binary, target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced'))

# Print the results
print(f"Final model trained on train_df with LOOCV Accuracy: {EndMotif_binary_LOOCV_accuracy:.4f}")

In [ ]:
# Combine train and test sets for LOOCV
Ratio_dataset_binary = subset_rows_by_target(Ratio_dataset, target_values)

# Test the train_model_loocv function
Ratio_binary_LOOCV_model, Ratio_binary_LOOCV_accuracy,_ = models.train_model_loocv(Ratio_dataset_binary, target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced'))

# Print the results
print(f"Final model trained on train_df with LOOCV Accuracy: {Ratio_binary_LOOCV_accuracy:.4f}")

In [ ]:

# NESTED CV
#EndMotif_binary_nestedCV_model, EndMotif_binary_nestedCV_accuracy = models.train_model_nested_cv(EndMotif_dataset_binary, target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced'), param_grid=param_grid_log_reg)

# Print the results
#print(f"Final model trained on train_df with LOOCV Accuracy: {EndMotif_binary_nestedCV_accuracy:.4f}")

In [ ]:
# Combine train and test sets for LOOCV
Arm_dataset_binary = subset_rows_by_target(Arm_df, target_values)

# Test the train_model_loocv function
Arm_binary_LOOCV_model, Arm_binary_LOOCV_accuracy,_ = models.train_model_loocv(Arm_dataset_binary, target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced'))

# Print the results
print(f"Final model trained on train_df with LOOCV Accuracy: {Arm_binary_LOOCV_accuracy:.4f}")

In [ ]:
# Combine train and test sets for LOOCV
Methylation_binary = subset_rows_by_target(Methylation, target_values)

# Test the train_model_loocv function
Methylation_binary_LOOCV_model, Methylation_binary_LOOCV_accuracy,_ = models.train_model_loocv(Methylation_binary, target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced'))

# Print the results
print(f"Final model trained on train_df with LOOCV Accuracy: {Methylation_binary_LOOCV_accuracy:.4f}")

# cfRNA

In [ ]:
cfRNA_file = '/Users/irf3irf3/Desktop/offline_workspace/Jessica/R code for bladder vs. healthy models/cfrna_tpm.tsv'

cfRNA = pd.read_csv(cfRNA_file, sep='\t', index_col=0)

cfRNA.rename(columns={'cohort': 'target'}, inplace=True)


print(cfRNA.shape)
display(cfRNA.head())

In [ ]:
# Replace 'your_file.txt' with the path to your file
protein_gene_file = '/Users/irf3irf3/Desktop/offline_workspace/ResearchCode2_in_transition/Ensemble_unique_ProteinCodig_genes_sorted.bed'

# Read the file with no header
protein_gene_df = pd.read_csv(protein_gene_file, sep='\t', header=None)

# Extract the 4th column (index 3) as a list
protein_gene = protein_gene_df[3].tolist()
protein_gene[:10]

In [ ]:
# Ensure 'target' is included in the subset
columns_to_keep = [col for col in protein_gene if col in cfRNA.columns] + ['target','batch']

# Subset the DataFrame
cfRNA = cfRNA[columns_to_keep]
print(cfRNA.shape)
display(cfRNA.head())

In [ ]:
cfRNA_allHealthy=cfRNA[(cfRNA['target'] == 'Healthy')].copy()

display(cfRNA_allHealthy.head())
print(cfRNA_allHealthy.shape)

In [ ]:
cfRNA_batch1_healthy = cfRNA[~((cfRNA['target'] == 'Healthy') & (cfRNA['batch'] == 2))]
cfRNA_batch1_healthy = cfRNA_batch1_healthy.drop(columns=['batch'])
cfRNA_batch2_healthy = cfRNA[~((cfRNA['target'] == 'Healthy') & (cfRNA['batch'] == 1))]
cfRNA_batch2_healthy = cfRNA_batch2_healthy.drop(columns=['batch'])
print(cfRNA_batch1_healthy.shape)
print(cfRNA_batch2_healthy.shape)

In [ ]:
display(cfRNA_batch1_healthy.head())

In [ ]:
import pandas as pd

def get_most_variable_genes(df, target_col='target', n_genes=10):
    """
    Get a DataFrame containing the most variable genes and the target column.

    Parameters:
    - df: DataFrame where genes are columns and samples are rows, except for the 'target' column.
    - target_col: Name of the column containing the class labels. Default is 'target'.
    - n_genes: Number of top variable genes to include. Default is 10.

    Returns:
    - A DataFrame containing the target column and the most variable genes.
    """
    # Exclude the target column to calculate variance only for genes
    gene_df = df.drop(columns=[target_col])
    
    # Calculate the variance for each gene
    variances = gene_df.var()
    
    # Identify the most variable genes
    most_variable_genes = variances.sort_values(ascending=False).head(n_genes).index.tolist()
    
    # Create a new DataFrame with the target column and the most variable genes
    result_df = df[[target_col] + most_variable_genes]
    
    return result_df

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os

def plot_pca(df, title, target_col='target', n_components=2, x_lim=None, y_lim=None, z_lim=None, color_map=None,label_point=False,legend_class_tittle='Class'):
    """
    Plot PCA of gene expression data by sample type.

    Parameters:
    - df: DataFrame where one column is the target (class labels) and others are features.
    - title: Title for the PCA plot.
    - target_col: Name of the column containing class labels. Default is 'target'.
    - n_components: Number of principal components to plot (2 or 3).
    - x_lim: Tuple (min, max) for x-axis limits. Default is None, showing the full range.
    - y_lim: Tuple (min, max) for y-axis limits. Default is None, showing the full range.
    - z_lim: Tuple (min, max) for z-axis limits if n_components=3. Default is None.
    - color_map: Optional dictionary mapping class names to colors.
    """
    # Dynamically separate target and features
    target = df[target_col].values
    features = df.drop(columns=[target_col]).values  # Drop the target column to get features

    # Log transformation
    features_log_transformed = np.log2(features.astype(float) + 1)

    # Scaling
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features_log_transformed)

    # PCA
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(features_scaled)

    # Plotting the PCA results with group labels
    if n_components == 2:
        plt.figure(figsize=(10, 8))
        for group in np.unique(target):
            idx = target == group
            color = color_map[group] if color_map and group in color_map else None
            plt.scatter(pca_result[idx, 0], pca_result[idx, 1], label=group, s=100, color=color)

        if label_point:
            # Label all points with their original indices
            for x, y, label in zip(pca_result[:, 0], pca_result[:, 1], df.index):
                plt.text(x, y, str(label), fontsize=8, ha='right', va='bottom', color='black')

        plt.xlabel('PC1')
        plt.ylabel('PC2')
    elif n_components == 3:
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        for group in np.unique(target):
            idx = target == group
            color = color_map[group] if color_map and group in color_map else None
            ax.scatter(pca_result[idx, 0], pca_result[idx, 1], pca_result[idx, 2], label=group, s=100, color=color)
        ax.set_xlabel('PC1')
        ax.set_ylabel('PC2')
        ax.set_zlabel('PC3')

    # Set axis limits if provided
    if n_components == 2:
        if x_lim is not None:
            plt.xlim(x_lim)
        if y_lim is not None:
            plt.ylim(y_lim)
    elif n_components == 3:
        if x_lim is not None:
            ax.set_xlim(x_lim)
        if y_lim is not None:
            ax.set_ylim(y_lim)
        if z_lim is not None:
            ax.set_zlim(z_lim)

   # plt.title(title)
    plt.legend(title=legend_class_tittle)


    cfRNA_PCA_dir = Output_dir +"_/cfRNA_PCA"  # Path to the new directory

    # Create the directory if it does not exist
    os.makedirs(cfRNA_PCA_dir, exist_ok=True)
    plot_filename = cfRNA_PCA_dir+'/'+legend_class_tittle+"_"+title+"_label_point_"+str(label_point)+".png"
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight')


    plt.show()



In [ ]:
plot_pca(cfRNA_allHealthy.drop(['target'],axis=1), 'All genes',target_col='batch',legend_class_tittle='batch')



In [ ]:
plot_pca(cfRNA_allHealthy.drop(['target'],axis=1), 'All genes',target_col='batch',legend_class_tittle='batch', label_point=True)



In [ ]:
plot_pca(cfRNA_batch1_healthy,  'All genes')

In [ ]:
plot_pca(get_most_variable_genes(cfRNA_batch1_healthy,n_genes=1000),  'top1000')


In [ ]:
def log_transform_and_standardize(df, target_column='target'):
    """
    Log-transforms and standardizes features, excluding the target column.
    """
    # Separate features and target
    features = df.drop(columns=[target_column])
    target = df[target_column]

    # Log transformation
    log_features = features.applymap(lambda x: np.log2(x + 1))

    # Standardization
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(log_features)

    # Convert back to DataFrame
    scaled_df = pd.DataFrame(scaled_features, columns=features.columns, index=df.index)

    # Add the target back
    scaled_df[target_column] = target

    return scaled_df

In [ ]:
cfRNA_batch1_healthy_standardized = log_transform_and_standardize(cfRNA_batch1_healthy)
cfRNA_batch2_healthy_standardized = log_transform_and_standardize(cfRNA_batch2_healthy)

In [ ]:

cfRNA_batch1_healthy_standardized_mostvariableGenes=get_most_variable_genes(cfRNA_batch1_healthy_standardized ,n_genes=1000)
# Test the train_model_loocv function
cfRNA_batch1_healthy_standardized_LOOCV_model, cfRNA_batch1_healthy_standardized_LogReg_LOOCV_auc,cfRNA_LogReg_oof_df = models.train_model_loocv(cfRNA_batch1_healthy_standardized_mostvariableGenes, target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced',max_iter=1000),save_figures_path=Output_dir+'_/LogReg_cfRNA_LOOCV_ROC', save_folder=Output_dir+'_/LogReg_cfRNA_LOOCV_ROC')


LOOCV_summary['cfRNA_batch1_healthy_standardized_LogReg_LOOCV_auc'] = cfRNA_batch1_healthy_standardized_LogReg_LOOCV_auc
# Print the results
#print(f"Final model trained on train_df with LOOCV Accuracy: {cfRNA_batch1_healthy_standardized_LOOCV_accuracy:.4f}")

In [ ]:
_,cfRNA_batch1_healthy_standardized_RF_LOOCV_auc,_= models.train_model_loocv(cfRNA_batch1_healthy_standardized_mostvariableGenes, target_name='target',model=RandomForestClassifier(random_state=0,n_estimators=2000, class_weight='balanced'),save_figures_path=Output_dir+'_/RF_cfRNA_LOOCV_ROC', save_folder=Output_dir+'_/RF_cfRNA_LOOCV_ROC')




LOOCV_summary['cfRNA_batch1_healthy_standardized_RF_LOOCV_auc'] = cfRNA_batch1_healthy_standardized_RF_LOOCV_auc


In [ ]:
_,cfRNA_batch1_healthy_standardized_SVM_LOOCV_auc,_ = models.train_model_loocv(cfRNA_batch1_healthy_standardized_mostvariableGenes, target_name='target',
    model=SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, class_weight='balanced'), 
    save_figures_path=Output_dir+'_/SVM_cfRNA_LOOCV_ROC', 
    save_folder=Output_dir+'_/SVM_cfRNA_LOOCV_ROC')

LOOCV_summary['cfRNA_batch1_healthy_standardized_SVM_LOOCV_auc'] = cfRNA_batch1_healthy_standardized_SVM_LOOCV_auc



In [ ]:

# # Test the train_model_loocv function
# cfRNA_batch2_healthy_LOOCV_model, cfRNA_batch2_healthy_LOOCV_accuracy,_ = models.train_model_loocv(get_most_variable_genes(cfRNA_batch2_healthy,n_genes=1000), target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced',max_iter=1000))

# # Print the results
# print(f"Final model trained on train_df with LOOCV Accuracy: {cfRNA_batch2_healthy_LOOCV_accuracy:.4f}")

In [ ]:
# # Test the train_model_loocv function
# cfRNA_batch2_healthy_standardized_LOOCV_model, cfRNA_batch2_healthy_standardized_LOOCV_accuracy,_ = models.train_model_loocv(get_most_variable_genes(cfRNA_batch2_healthy_standardized ,n_genes=1000), target_name='target',model=LogisticRegression(random_state=0, class_weight='balanced',max_iter=1000))

# # Print the results
# print(f"Final model trained on train_df with LOOCV Accuracy: {cfRNA_batch2_healthy_standardized_LOOCV_accuracy:.4f}")

In [ ]:
cfRNA_batch1_healthy_standardized.head()

In [ ]:
cfRNA_batch1_healthy_standardized_mostvariableGenes_train, cfRNA_batch1_healthy_standardized_mostvariableGenes_test = preprocessing.stratified_train_test_split(cfRNA_batch1_healthy_standardized_mostvariableGenes, test_size=TEST_SIZE)



In [ ]:
_,cfRNA_batch1_healthy_standardized_SVM_auc = models.train_model(
    cfRNA_batch1_healthy_standardized_mostvariableGenes_train, cfRNA_batch1_healthy_standardized_mostvariableGenes_test,
    target_name='target', 
    model=SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=0), 
    param_grid=param_grid_svm,  # Ensure you define param_grid for SVM
    cv=5, 
    search_method='grid',
    save_figures_path=Output_dir+'_/SVC_cfRNA_ROC', 
    save_folder=Output_dir+'_/SVC_cfRNA_ROC'
)

Train_test_summary['cfRNA_batch1_healthy_standardized_SVM_auc'] = cfRNA_batch1_healthy_standardized_SVM_auc


In [ ]:
_,cfRNA_batch1_healthy_standardized_LogReg_auc = models.train_model(cfRNA_batch1_healthy_standardized_mostvariableGenes_train, cfRNA_batch1_healthy_standardized_mostvariableGenes_test, model=LogisticRegression(random_state=0, class_weight='balanced'), 
            param_grid=param_grid_log_reg, 
            cv=5, 
   
            search_method='grid',
            save_figures_path=Output_dir+'_/Log_reg_cfRNA_ROC', 
            save_folder=Output_dir+'_/Log_reg_cfRNA_ROC'
                   
                  )

Train_test_summary['cfRNA_batch1_healthy_standardized_LogReg_auc'] = cfRNA_batch1_healthy_standardized_LogReg_auc




In [ ]:
_, cfRNA_batch1_healthy_standardized_RF_auc = models.train_model(cfRNA_batch1_healthy_standardized_mostvariableGenes_train, cfRNA_batch1_healthy_standardized_mostvariableGenes_test, model = RandomForestClassifier(random_state=0, n_estimators=2000, class_weight='balanced'), param_grid=param_grid_rf, cv=5, scoring='roc_auc_micro', search_method='grid',
                save_figures_path=Output_dir+'_/RF_cfRNA_ROC', 
                save_folder=Output_dir+'_/RF_cfRNA_ROC')

Train_test_summary['cfRNA_batch1_healthy_standardized_RF_auc'] = cfRNA_batch1_healthy_standardized_RF_auc



In [ ]:
Train_test_summary

In [ ]:
plotting_utils.plot_auc_heatmap(Train_test_summary,save_figure_path=Output_dir+'_/Train_test_summary.png')

In [ ]:
Train_test_summary_rename = {k: v for k, v in Train_test_summary.items() if "CNA" not in k}

Train_test_summary_rename= preprocessing.rename_keys(Train_test_summary_rename, {'Arm':'CNA (Arm)','Ratio':'Fragment Length','EndMotif':'End Motif','LogReg':'Logistic Regression','RF':'Random Forest','SVM':'Support Vector Machine'})

plotting_utils.plot_auc_heatmap(Train_test_summary_rename,save_figure_path=Output_dir+'_/Train_test_summary_rename.png')


In [ ]:
LOOCV_summary

In [ ]:
plotting_utils.plot_auc_heatmap(LOOCV_summary, save_figure_path=Output_dir+'_/LOOCV_summary.png')

In [ ]:
LOOCV_summary_rename= preprocessing.rename_keys(LOOCV_summary, {'Arm':'CNA (Arm)','Ratio':'Fragment Length','EndMotif':'End Motif','LogReg':'Logistic Regression','RF':'Random Forest','SVM':'Support Vector Machine'})
plotting_utils.plot_auc_heatmap(LOOCV_summary_rename,save_figure_path=Output_dir+'_/LOOCV_summary_rename.png')


# LOD

In [ ]:
end_motifs_mix = pd.read_csv('/Users/irf3irf3/Desktop/offline_workspace/data/tissue_of_origin/Jessica_LOD/data/motif_counts_mix.txt',sep='\t',index_col=0)

end_motifs_mix.index.name= None

display(end_motifs_mix.head())




In [ ]:
end_motifs_mix_t = preprocessing.preprocess_dataframe(end_motifs_mix)

display(end_motifs_mix_t.head())

In [ ]:
end_motifs_mix_t = preprocessing.normalize_features_by_sample(end_motifs_mix_t)

display(end_motifs_mix_t)

In [ ]:
# Extract features (excluding 'target')
X_test = end_motifs_mix.drop(columns=['target'])

# Extract ground truth labels
y_true = end_motifs_mix['target']

# Make predictions
y_pred = EndMotif_logreg_trained_model.predict(X_test)

# Get predicted probabilities
y_pred_proba = EndMotif_LogReg_LOOCV_model.predict_proba(X_test)

# Get the probability of the predicted class
predicted_proba = y_pred_proba.max(axis=1)  # Take max probability corresponding to predicted class

# Create a DataFrame to store results
results_df = end_motifs_mix.copy()
results_df['predicted_target'] = y_pred
results_df['predicted_probability'] = predicted_proba  # Store max probability
results_df['correct_prediction'] = results_df['target'] == results_df['predicted_target']

# Get correctly predicted indices
correctly_predicted_indices = results_df.index[results_df['correct_prediction']]

# Display the results
display(results_df)


In [ ]:
import sys
sys.exit()

# Biological Significance
## End motif

In [ ]:
EndMotif_feature_importance.head()

In [ ]:
EndMotif_feature_importance.tail()

In [ ]:
#train_df, test_df
train_df.head()

In [ ]:
import Biological_significace as Biosig
import os
import pandas as pd
from statsmodels.stats.multitest import multipletests

def save_plots_and_create_summary(df, feature_df, target_name='target', save_dir=None):
    """
    This function generates and saves box plots for each feature listed in the 'Feature' column of feature_df,
    saves the plots using the plot_feature_boxplot_with_anova function, and creates a summary DataFrame containing
    the F-statistic, p-value, and Benjamini-Hochberg (BH) corrected p-values for each feature.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the features and target.
    feature_df (pd.DataFrame): A DataFrame with a column named 'Feature' listing the features to analyze.
    target_name (str): The name of the target column in df, default is 'target'.
    save_dir (str): The directory where the plots and summary will be saved. This is a required argument.
    
    Returns:
    pd.DataFrame: A DataFrame containing the F-statistic, p-value, and BH-corrected p-value for each feature.
    """
    if not save_dir:
        raise ValueError("The save_dir argument is required.")
    
    # Check if the directory already exists, if so, raise an error
    if os.path.exists(save_dir):
        raise FileExistsError(f"The directory '{save_dir}' already exists. Please choose a different directory or remove it.")
    
    # Create the save directory
    os.makedirs(save_dir)
    
    summary = []
    class_order= ['Bladder','Prostate','RCC','Healthy']
    
    for feature in feature_df['Feature']:
        # Define the save path for each plot
        save_path = os.path.join(save_dir, f"{feature}_boxplot.png")
        
        # Generate and save the plot, and get the F-statistic and p-value
        f_stat, p_value = Biosig.plot_feature_boxplot_with_kruskal(df, feature, target_name=target_name, save_path=save_path, class_order= class_order)
        
        # Append the results to the summary list
        summary.append({
            'Feature': feature,
            'F-statistic': f_stat,
            'p-value': p_value
        })
    
    # Create a summary DataFrame from the list
    summary_df = pd.DataFrame(summary)
    
    # Perform Benjamini-Hochberg correction
    _, bh_corrected_pvals, _, _ = multipletests(summary_df['p-value'], method='fdr_bh')
    
    # Add the BH-corrected p-values to the DataFrame
    summary_df['BH-corrected p-value'] = bh_corrected_pvals
    
    # Save the summary DataFrame as a CSV file in the save directory
    summary_csv_path = os.path.join(save_dir, 'summary.csv')
    summary_df.to_csv(summary_csv_path, index=False)
    
    # Return the summary DataFrame
    return summary_df


In [ ]:

Biosig.plot_feature_boxplot_with_anova(train_df, 'EndMotif_AATA')
Biosig.plot_feature_boxplot_with_anova(train_df, 'EndMotif_AATA',class_order= ['Bladder','Prostate','RCC','Healthy'])
Biosig.plot_feature_boxplot_with_anova(train_df, 'EndMotif_AGTG',class_order= ['Bladder','Prostate','RCC','Healthy'])
Biosig.plot_feature_boxplot_with_anova(train_df, 'EndMotif_CGAA',class_order= ['Bladder','Prostate','RCC','Healthy'])
Biosig.plot_feature_boxplot_with_anova(train_df, 'EndMotif_TCTA',class_order= ['Bladder','Prostate','RCC','Healthy'])
Biosig.plot_feature_boxplot_with_anova(train_df, 'EndMotif_TGTG',class_order= ['Bladder','Prostate','RCC','Healthy'])

In [ ]:
EndMotif_Bio = save_plots_and_create_summary(train_df, EndMotif_feature_importance,  save_dir=Output_dir+'/EdMotif_traindf_KruskalWallis/')
display(EndMotif_Bio.head())